In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# dand wiki

In [159]:
# URL of the webpage containing the spell list
base_url = 'https://www.dandwiki.com'
spells_url = f'{base_url}/wiki/5e_SRD:Spells'

# Send a GET request to the webpage
response = requests.get(spells_url)
response.raise_for_status() 

In [160]:
soup = BeautifulSoup(response.content, 'html.parser')

In [173]:
def get_spell_details(spell_url):
    damage_types = ['piercing, bludgeoning, slashing, cold, fire, lightning, thunder, poison, acid, necrotic, radiant, force, psychic']
    response = requests.get(base_url + spell_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    spell_details = {}
    if soup.find('caption'):
        spell_name = soup.find('caption').text.strip()
        spell_details['Name'] = spell_name
    
        labels = soup.find('table', class_='d20').find('td').text.strip().split(' ')
        if '(ritual)' in labels:
            spell_details['Ritual'] = True
        else:
            spell_details['Ritual'] = False

        if 'cantrip' in labels:
            spell_details['Level'] = 'Cantrip'
            spell_details['School'] = labels[0]
        else:
            spell_details['Level'] = labels[0][0]
            spell_details['School'] = labels[1]
            
        spell_details_table = soup.find('table', class_='d20').find_all('tr')
        for row in spell_details_table:
            cells = row.find_all(['th', 'td'])
            if len(cells) == 2:
                key = cells[0].text.strip().rstrip(':')
                value = cells[1].text.strip()
                spell_details[key] = value
        description_paragraphs = soup.find_all('p')
        description = '\n'.join([p.text.strip() for p in description_paragraphs if not p.find('a', title="Main Page")])
    
    
        spell_details['Description'] = description
    
        damage_occurrences = len(re.findall('damage', description))
        init_index = 0
        damage_type_list = []
        for i in range(damage_occurrences):
            index_of_damage = description[init_index:].find('damage')
            if index_of_damage != -1:
                last_space_before_damage = description.rfind(" ", 0, index_of_damage-1)
                damage_type = description[last_space_before_damage+1:index_of_damage-1]
                if damage_type in damage_types:
                    damage_type_list.append(damage_type)
                init_index = index_of_damage
        if damage_type_list:
            spell_details['Damage Type'] = set(damage_type_list)
        else:
            spell_details['Damage Type'] = 'Non-damaging'
            
        index_of_attack = description.find('spell attack')
        if index_of_attack != -1:
            last_space_before_attack = description.rfind(" ", 0, index_of_attack-1)
            attack_type = description[last_space_before_attack+1:index_of_attack-1]
            spell_details['Attack Type'] = attack_type
        else:
            spell_details['Attack Type'] = None
        
        index_of_save = description.find('saving throw')
        if index_of_save != -1:
            last_space_before_save = description.rfind(" ", 0, index_of_save-1)
            save_type = description[last_space_before_save+1:index_of_save-1]
            spell_details['Saving Throw'] = save_type
        else:
            spell_details['Saving Throw'] = None
    
        
        return spell_details

In [175]:
spells_list = []
a_tag_list = []
ul_list = []
for ul in soup.find_all('ul')[5:15]: # find all unordered lists # Need 5 to 14 which are the spells
    ul_list.append (ul)
    for li in ul.find_all('li'): # find all list items per unordered list
        a_tag = li.find('a')
        a_tag_list.append(a_tag)
        if a_tag and 'href' in a_tag.attrs:
            spell_url = a_tag['href'] # this gets the relative url attached to a_tag
            spell_details = get_spell_details(spell_url)
            spells_list.append(spell_details)
            time.sleep(0.5) # be polite to the server

spells_list_filtered = [entry for entry in spells_list if entry is not None]
spells_df = pd.DataFrame(spells_list_filtered)

# Save the DataFrame to a CSV file
spells_df.to_csv('5e_spells.csv', index=False)
# spells_df = pd.DataFrame(spells_list)
#spells_df.to_csv('5e_spells.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'keys'

# Wikidot

* td - table data
* th - table header
* li - list
* tr - data row
* a - anchor tag - used to create hyperlinks
* p - defines paragraph
* strong - emphasis
* br - line break

In [32]:
# URL of the webpage containing the spell list
base_url = 'http://dnd5e.wikidot.com'
spells_url = f'{base_url}/spells'

# Send a GET request to the webpage
response = requests.get(spells_url)
response.raise_for_status() 

In [33]:
soup = BeautifulSoup(response.content, 'html.parser')

In [34]:
soup

<!DOCTYPE html>

<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Spells - DND 5th Edition</title>
<script src="http://d3g0gp89917ko0.cloudfront.net/v--b466fa696223/common--javascript/init.combined.js" type="text/javascript"></script>
<script type="text/javascript">
        var URL_HOST = 'www.wikidot.com';
        var URL_DOMAIN = 'wikidot.com';
        var USE_SSL =  true ;
        var URL_STATIC = 'http://d3g0gp89917ko0.cloudfront.net/v--b466fa696223';
        // global request information
        
        var WIKIREQUEST = {};
        WIKIREQUEST.info = {};
        
        WIKIREQUEST.info.domain = "dnd5e.wikidot.com";
        WIKIREQUEST.info.siteId = 1475899;
        WIKIREQUEST.info.siteUnixName = "dnd5e";
        WIKIREQUEST.info.categoryId = 11887298;
        WIKIREQUEST.info.themeId = 2386862;
        WIKIREQUEST.info.requestPageName = "spells";
        OZONE.request.timestamp = 1718322239;
        OZONE.request.date = new Date();
        WIK

In [8]:
spell_levels = ['Cantrip', 1,2,3,4,5,6,7,8,9]
ct = 0
for level_table in soup.find_all('table'):
    spell_level = spell_levels[ct]
    for a_tag in level_table.find_all('a'):
        if a_tag and 'href' in a_tag.attrs:
            spell_url = a_tag['href']
            spell_details = get_spell_details(spell_url)
            spells_list.append(spell_details)
            time.sleep(0.5) # be polite to the server
        

[<table class="wiki-content-table">
 <tr>
 <th>Spell Name</th>
 <th>School</th>
 <th>Casting Time</th>
 <th>Range</th>
 <th>Duration</th>
 <th>Components</th>
 </tr>
 <tr>
 <td><a href="/spell:acid-splash">Acid Splash</a></td>
 <td><em>Conjuration</em></td>
 <td>1 Action</td>
 <td>60 Feet</td>
 <td>Instantaneous</td>
 <td>V, S</td>
 </tr>
 <tr>
 <td><a href="/spell:blade-ward">Blade Ward</a></td>
 <td><em>Abjuration</em></td>
 <td>1 Action</td>
 <td>Self</td>
 <td>1 round</td>
 <td>V, S</td>
 </tr>
 <tr>
 <td><a href="/spell:booming-blade">Booming Blade</a></td>
 <td><em>Evocation</em></td>
 <td>1 Action</td>
 <td>Self (5-foot radius)</td>
 <td>1 round</td>
 <td>S, M</td>
 </tr>
 <tr>
 <td><a href="/spell:chill-touch">Chill Touch</a></td>
 <td><em>Necromancy</em></td>
 <td>1 Action</td>
 <td>120 feet</td>
 <td>1 round</td>
 <td>V, S</td>
 </tr>
 <tr>
 <td><a href="/spell:control-flames">Control Flames</a></td>
 <td><em>Transmutation</em></td>
 <td>1 Action</td>
 <td>60 Feet</td>
 <td>I

In [30]:
spell_url

'/spell:acid-splash'

In [29]:
print(base_url + spell_url[1:])
response = requests.get(base_url + spell_url[1:])

http://dnd5e.wikidot.com/spell:acid-splash


In [36]:
# base_url = 'http://dnd5e.wikidot.com/'
# spells_url = f'{base_url}/spells'
# # Send a GET request to the webpage
# response = requests.get(spells_url)
# response.raise_for_status() 
# soup = BeautifulSoup(response.content, 'html.parser')

level_table = soup.find_all('table')[0]
a = level_table.find('a')
spell_url = a['href']
response = requests.get(base_url + spell_url)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

IndexError: list index out of range

In [57]:
def get_spell_details(base_and_spell_url):

    response = requests.get()
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    source = soup.find_all('p')[0].text[8:]
    text = soup.find_all('p')[1].text.split(' ')
    if 'cantrip' == text[1]:
        school = text[0]
        level = text[1]
    else:
        school = text[1]
        level = text[0]
    
    if 'ritual' in text:
        ritual = True
    else:
        ritual = False
    
    time = soup.find_all('p')[2].text.split('\n')[0][14:]
    range = soup.find_all('p')[2].text.split('\n')[1][7:]
    comp = soup.find_all('p')[2].text.split('\n')[2][12:]
    duration = soup.find_all('p')[2].text.split('\n')[3][10:]
    description = soup.find_all('p')[3].text+soup.find_all('p')[4].text
    
    # pattern = r'\d+d\d+'
    
    # Find all occurrences of the pattern in the paragraph
    # dice_notations = re.findall(pattern, paragraph)
    
    if 'saving throw' in description:
        save = True
        indices = [index-1 for index, value in enumerate(description.split(' ')) if value == 'saving']
        save_type = [description.split(' ')[index] for index in indices]
        
    spell_lists = soup.find_all('p')[5].text[13:]

    

1 action
60 feet
V, S
Instantaneous
